In [ ]:
import os
import torch
import pandas as pd
from tqdm import tqdm
import numpy as np

tqdm.pandas()

In [ ]:
embs = [torch.load(f'./parallel_shards_embeddings_bert/'+f) for f in sorted(os.listdir('parallel_shards_embeddings_bert/'))]
final_emb = {'index':[], 'embeddings':[]}

for emb in embs:
    final_emb['index'].extend(emb['index'])
    final_emb['embeddings'].append(emb['embeddings'])
    
final_emb['embeddings'] = torch.cat(final_emb['embeddings'])
final_emb = dict(zip(final_emb['index'], final_emb['embeddings']))

def get_sim(x, y):
    cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)
    return cos(final_emb[x], final_emb[y]).item()

In [ ]:
dfs = [pd.read_parquet('./parallel_shards/'+f) for f in sorted(os.listdir('parallel_shards/'))]
df = pd.concat(dfs)
df = df[df['id'].isin(final_emb.keys())]
df_pairs= df.merge(df, on=['username','url'], how='inner')
df_pairs= df_pairs[df_pairs['id_x']<df_pairs['id_y']]

In [ ]:
df_pairs['similarity'] = df_pairs.apply(lambda x: get_sim(x['id_x'], x['id_y']), axis=1)

In [ ]:
m = df_pairs[['id_x', 'id_y', 'url', 'username']]
tfidf = pd.read_parquet('tfidf.parquet')
df_pairs['tfidf_similarity'] = m.merge(tfidf, on=['id_x', 'id_y', 'url', 'username'], how='left')['tfidf_similarity']

In [ ]:
df_pairs.to_parquet('parallel_tweet_it0.parquet')
torch.save(final_emb, 'tweet_it0_bert.pt')